In [1]:
!pip install streamlit pyngrok sentence-transformers pymupdf


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 84.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 58.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 74.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 105.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 84.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 53.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.

In [2]:
app_code = """
import streamlit as st
import fitz  # PyMuPDF
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer('all-MiniLM-L6-v2')

st.set_page_config(page_title="HR Candidate Evaluator", layout="centered")
st.title("🤖 HR Candidate Screening Assistant")

# Step 1: Select job requirements
with st.expander("Step 1: Select Required Attributes"):
    skills = st.multiselect("Required Skills", [
        "Python", "Machine Learning", "Communication", "Teamwork", "Leadership",
        "NLP", "Data Analysis", "SQL", "Java", "AWS", "Project Management"
    ])
    experience = st.slider("Years of Experience Required", 0, 20, 2)
    degree = st.selectbox("Minimum Degree Required", ["Any", "Bachelor's", "Master's", "PhD"])

# Step 2: Input job description
job_desc = st.text_area("Paste the Job Description Here", height=200)

# Step 3: Upload CV
cv_file = st.file_uploader("Upload Candidate's CV (PDF only)", type=['pdf'])

def extract_text_from_pdf(uploaded_file):
    text = ""
    with fitz.open(stream=uploaded_file.read(), filetype="pdf") as doc:
        for page in doc:
            text += page.get_text()
    return text

# Step 4: Match Candidate
if st.button("🧠 Evaluate Candidate"):
    if not job_desc or not cv_file:
        st.warning("Please upload a CV and provide a job description.")
    else:
        with st.spinner("Analyzing candidate..."):
            # Extract CV text
            cv_text = extract_text_from_pdf(cv_file)

            # Combine selected attributes into context
            attr_string = " ".join(skills) + f" {experience} years experience {degree} degree"
            combined_job_text = job_desc + " " + attr_string

            # Create embeddings and compare
            embeddings = model.encode([combined_job_text, cv_text], convert_to_tensor=True)
            similarity = util.pytorch_cos_sim(embeddings[0], embeddings[1]).item()
            match_percentage = round(similarity * 100, 2)

            # Decision
            if match_percentage > 70:
                decision = "✅ Recommend for Interview"
            elif match_percentage > 50:
                decision = "⚠️ Maybe - Needs Manual Review"
            else:
                decision = "❌ Not Recommended"

            # Output
            st.subheader("🔍 Match Result")
            st.write(f"**Match Score:** {match_percentage}%")
            st.write(f"**Decision:** {decision}")
"""
with open("app.py", "w") as f:
    f.write(app_code)


In [3]:
from pyngrok import ngrok

# Kill any existing tunnels
ngrok.kill()

# Open a new tunnel on port 8501
public_url = ngrok.connect(port=8501)
print(f"🌐 Open your app here: {public_url}")

# Start the streamlit app
!streamlit run app.py &>/content/logs.txt &


ERROR:pyngrok.process.ngrok:t=2025-07-25T17:01:48+0000 lvl=eror msg="failed to reconnect session" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"
ERROR:pyngrok.process.ngrok:t=2025-07-25T17:01:48+0000 lvl=eror msg="session closing" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"
ERROR:pyngrok.process.ngrok:t=2025-07-25T17:01:48+0000 lvl=eror msg="terminating with error" obj=app err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your aut

PyngrokNgrokError: The ngrok process errored on start: authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n.

In [4]:
from pyngrok import ngrok

# Set your ngrok auth token
ngrok.set_auth_token("30NLFFTNXK8XLUOe56zXf8gEgdr_7TgTE4TYAw4Hb4AEZRiiT")


In [5]:
# Kill any existing tunnels (if needed)
ngrok.kill()

# Create a new tunnel on port 8501
public_url = ngrok.connect(port=8501)
print(f"🌐 Open your app here: {public_url}")

# Start the Streamlit app
!streamlit run app.py &>/content/logs.txt &


PyngrokNgrokHTTPError: ngrok client exception, API returned 400: {"error_code":102,"status_code":400,"msg":"invalid tunnel configuration","details":{"err":"yaml: unmarshal errors:\n  line 1: field port not found in type config.HTTPv2Tunnel"}}


In [6]:
# Kill any existing tunnels
ngrok.kill()

# Corrected tunnel setup for ngrok v3 (bind_tls is important!)
public_url = ngrok.connect(8501, bind_tls=True)
print(f"🌐 Open your app here: {public_url}")

# Start the Streamlit app
!streamlit run app.py &>/content/logs.txt &


ERROR:pyngrok.process.ngrok:t=2025-07-25T17:07:02+0000 lvl=eror msg="failed to reconnect session" obj=tunnels.session err="authentication failed: Your account is limited to 1 simultaneous ngrok agent sessions.\nYou can run multiple simultaneous tunnels from a single agent session by defining the tunnels in your agent configuration file and starting them with the command `ngrok start --all`.\nRead more about the agent configuration file: https://ngrok.com/docs/secure-tunnels/ngrok-agent/reference/config\nYou can view your current agent sessions in the dashboard:\nhttps://dashboard.ngrok.com/agents\r\n\r\nERR_NGROK_108\r\n"
ERROR:pyngrok.process.ngrok:t=2025-07-25T17:07:02+0000 lvl=eror msg="session closing" obj=tunnels.session err="authentication failed: Your account is limited to 1 simultaneous ngrok agent sessions.\nYou can run multiple simultaneous tunnels from a single agent session by defining the tunnels in your agent configuration file and starting them with the command `ngrok st

PyngrokNgrokError: The ngrok process errored on start: authentication failed: Your account is limited to 1 simultaneous ngrok agent sessions.\nYou can run multiple simultaneous tunnels from a single agent session by defining the tunnels in your agent configuration file and starting them with the command `ngrok start --all`.\nRead more about the agent configuration file: https://ngrok.com/docs/secure-tunnels/ngrok-agent/reference/config\nYou can view your current agent sessions in the dashboard:\nhttps://dashboard.ngrok.com/agents\r\n\r\nERR_NGROK_108\r\n.

In [7]:
ngrok.kill()



In [8]:
public_url = ngrok.connect(8501, bind_tls=True)
print(f"🌐 Open your app here: {public_url}")

!streamlit run app.py &>/content/logs.txt &


🌐 Open your app here: NgrokTunnel: "https://85ac37d3d19a.ngrok-free.app" -> "http://localhost:8501"
